In [1]:
!pip3 install pandas numpy datasets huggingface_hub --upgrade

Requirement already up-to-date: pandas in /home/sun/.local/lib/python3.8/site-packages (2.0.3)
Requirement already up-to-date: numpy in /home/sun/.local/lib/python3.8/site-packages (1.24.4)
Requirement already up-to-date: datasets in /home/sun/.local/lib/python3.8/site-packages (2.14.5)
Requirement already up-to-date: huggingface_hub in /home/sun/.local/lib/python3.8/site-packages (0.18.0)


In [2]:
import pandas as pd
import numpy as np
import os
import json

BASE_PATH = "/home/sun/Downloads/tg_challenge/"
# from google.colab import drive
# drive.mount('/content/drive')

# Data

In [3]:
PATH_TO_JSON_FILE = os.path.join(BASE_PATH, 'language_identification_dataset_100_with_messages_and_regexps.json')

with open(PATH_TO_JSON_FILE, 'r') as f:
    data = json.load(f)

In [4]:
list(data.keys())

['TGLANG_LANGUAGE_ABAP',
 'TGLANG_LANGUAGE_ACTIONSCRIPT',
 'TGLANG_LANGUAGE_ADA',
 'TGLANG_LANGUAGE_APACHE_GROOVY',
 'TGLANG_LANGUAGE_APPLESCRIPT',
 'TGLANG_LANGUAGE_ASP',
 'TGLANG_LANGUAGE_ASSEMBLY',
 'TGLANG_LANGUAGE_AUTOHOTKEY',
 'TGLANG_LANGUAGE_AWK',
 'TGLANG_LANGUAGE_BASIC',
 'TGLANG_LANGUAGE_BATCH',
 'TGLANG_LANGUAGE_BISON',
 'TGLANG_LANGUAGE_C',
 'TGLANG_LANGUAGE_CLOJURE',
 'TGLANG_LANGUAGE_CMAKE',
 'TGLANG_LANGUAGE_COBOL',
 'TGLANG_LANGUAGE_COFFESCRIPT',
 'TGLANG_LANGUAGE_COMMON_LISP',
 'TGLANG_LANGUAGE_CPLUSPLUS',
 'TGLANG_LANGUAGE_CRYSTAL',
 'TGLANG_LANGUAGE_CSHARP',
 'TGLANG_LANGUAGE_CSS',
 'TGLANG_LANGUAGE_CSV',
 'TGLANG_LANGUAGE_D',
 'TGLANG_LANGUAGE_DART',
 'TGLANG_LANGUAGE_DOCKER',
 'TGLANG_LANGUAGE_ELIXIR',
 'TGLANG_LANGUAGE_ELM',
 'TGLANG_LANGUAGE_ERLANG',
 'TGLANG_LANGUAGE_FORTH',
 'TGLANG_LANGUAGE_FORTRAN',
 'TGLANG_LANGUAGE_FSHARP',
 'TGLANG_LANGUAGE_GAMS',
 'TGLANG_LANGUAGE_GO',
 'TGLANG_LANGUAGE_GRAPHQL',
 'TGLANG_LANGUAGE_HASKELL',
 'TGLANG_LANGUAGE_HTML',
 'TGL

In [5]:
new_data = dict()
for key in data:
    if len(data[key]) == 1:
        new_data[key] = [data[key][0]]
    else:
        new_data[key] = [data[key]]

In [6]:
len(new_data['TGLANG_LANGUAGE_BASIC'][0][1])
# new_data.keys()

50

In [7]:
# data_for_df = [{"class": key, "text": value[0][i]} for key, value in new_data.items() for i in range(len(value[0]))]
data_for_df = []

for key, value in new_data.items():
    for i in range(len(value[0])):
        if key == 'TGLANG_LANGUAGE_BASIC':
            for j in range(len(value[0][i])):
                data_for_df.append({'class': key, 'text': value[0][i][j]})
        else:
            data_for_df.append({'class': key, 'text': value[0][i]})

In [8]:
data_for_modeling = pd.DataFrame(data_for_df).sample(frac=1)

# Load data

In [9]:
modeling_data_file = os.path.join(BASE_PATH, "data_for_modeling.json")
data_for_modeling.to_json(modeling_data_file)
data = pd.read_json(modeling_data_file)

In [10]:
data['class'].value_counts()

class
TGLANG_LANGUAGE_OTHER         5000
TGLANG_LANGUAGE_REGEX         2990
TGLANG_LANGUAGE_MARKDOWN       100
TGLANG_LANGUAGE_CLOJURE        100
TGLANG_LANGUAGE_FORTRAN        100
                              ... 
TGLANG_LANGUAGE_PROTOBUF       100
TGLANG_LANGUAGE_BATCH          100
TGLANG_LANGUAGE_DOCKER         100
TGLANG_LANGUAGE_TYPESCRIPT     100
TGLANG_LANGUAGE_NGINX           15
Name: count, Length: 85, dtype: int64

In [11]:
IMBALANCED_CLASSES = [
    "TGLANG_LANGUAGE_REGEX",
    "TGLANG_LANGUAGE_OTHER",
]

balanced_based = data[~data['class'].isin(IMBALANCED_CLASSES)]

In [12]:
balanced_dfs = [data[data['class'] == CLASS].sample(100, random_state=42) for CLASS in IMBALANCED_CLASSES]

In [13]:
balanced_data = pd.concat([balanced_based, *balanced_dfs], axis=0)

In [14]:
balanced_data.head()

,class,text
6718,TGLANG_LANGUAGE_SHELL,#!/bin/bash\n\nprintCommandHelp() {\n echo ...
7652,TGLANG_LANGUAGE_VALA,#!/usr/bin/vala\nvoid notify(string message) {...
6959,TGLANG_LANGUAGE_SOLIDITY,// SPDX-License-Identifier: Unlicense\npragma ...
6011,TGLANG_LANGUAGE_QML,/*********************************************...
112,TGLANG_LANGUAGE_ACTIONSCRIPT,package com\r\n{\r\n\timport com.utils.Utils;\...


In [15]:
# data[data['class'] == 'TGLANG_LANGUAGE_CPLUSPLUS'].text.str.contains("//").astype(int)

# all(isinstance(value, str) for value in data['text'])

# Modeling

In [16]:
!pip3 install scikit-learn

In [17]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, PredefinedSplit
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
### Splitting into folds
from typing import List
from tqdm import tqdm

K_FOLDS = 5

In [19]:



# evaluation utils
def generate_folding(k_folds: int, df: pd.DataFrame) -> List[int]:

    len_folding = df.shape[0] // k_folds
    all_folding = []
    for i in range(k_folds):

        if i != k_folds - 1:
            all_folding += [i + 1 for _ in range(len_folding)]
        else:
            len_folds_computed = df.shape[0] - len(all_folding)
            all_folding += [i + 1 for _ in range(len_folds_computed)]
    return all_folding




In [20]:
def get_test_fold_for_iteration(fold_indices, test_fold_num):
    return [0 if f == test_fold_num else -1 for f in fold_indices]

In [21]:
### Refactor
def train_multiple_folds(n_folds: int, df: pd.DataFrame, vectorizer: object) -> List[float]:

    X = vectorizer.fit_transform(df['text'])
    y = df['class']

    test_folding = generate_folding(k_folds=n_folds, df=df)

    clf = MultinomialNB()
    all_scores = []

    for fold_num in tqdm(range(n_folds)):  # As we have 4 folds
        current_test_fold = get_test_fold_for_iteration(test_folding, fold_num)
        ps = PredefinedSplit(current_test_fold)
        scores = cross_val_score(clf, X, y, cv=ps, scoring='f1_macro')
        all_scores.extend(scores)

    return all_scores

In [22]:
data['text'].head()

14399                                   ^[0-9]+(,[0-9]+)*$
11037                               نم مستغفراً \n11:11م 💙
9953     🎊🌟 2️⃣0️⃣2️⃣1️⃣🌟 🎊\n\n🎉 *HOT SELLING*🎉\n\n 🌈 *...
6718     #!/bin/bash\n\nprintCommandHelp() {\n    echo ...
14170                                  ^([^:])+\\.([^:])+$
Name: text, dtype: object

In [23]:
vectorizer = TfidfVectorizer(lowercase=False)

In [24]:
X = vectorizer.fit_transform(data['text'])
y = data['class']

In [25]:
X.shape, len(data), len(y)

((16205, 764896), 16205, 16205)

In [26]:
vectorizer.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': False,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 1),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': None,
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'use_idf': True,
 'vocabulary': None}

In [27]:
vectorizer.idf_

array([4.42404054, 5.45138981, 6.336428  , ..., 9.99998964, 9.99998964,
       9.99998964])

In [28]:
from sklearn.metrics import f1_score

clf = MultinomialNB()
# usual traning would be here
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=42)


In [29]:
mnb = clf.fit(train_X, train_y)
mnb

MultinomialNB()

In [30]:
mnb.feature_count_.shape

(85, 764896)

In [31]:
mnb.feature_log_prob_.shape

(85, 764896)

In [32]:
mnb.alpha

1.0

In [33]:
mnb.score(val_X, val_y)

0.41609081934846986

In [34]:
from skl2onnx import to_onnx
from skl2onnx.common.data_types import FloatTensorType, Int64TensorType, DoubleTensorType
to_onnx?

Signature:
to_onnx(
    model,
    X=None,
    name=None,
    initial_types=None,
    target_opset=None,
    options=None,
    white_op=None,
    black_op=None,
    final_types=None,
    dtype=None,
    naming=None,
    model_optim=True,
    verbose=0,
)
Docstring:
Calls :func:`convert_sklearn` with simplified parameters.

:param model: model to convert
:param X: training set, can be None, it is used to infered the
    input types (*initial_types*)
:param initial_types: if X is None, then *initial_types* must be
    defined
:param target_opset: conversion with a specific target opset
:param options: specific options given to converters
    (see :ref:`l-conv-options`)
:param name: name of the model
:param white_op: white list of ONNX nodes allowed
    while converting a pipeline, if empty, all are allowed
:param black_op: black list of ONNX nodes allowed
    while converting a pipeline, if empty, none are blacklisted
:param final_types: a python list. Works the same way as initial_types

In [35]:
cols = X.shape[1]
cols

764896

In [36]:
initial_types = [('float_input', FloatTensorType([None, cols]))]

In [37]:
onnx_clf = to_onnx(clf, initial_types=initial_types)

In [38]:
onnx_path = os.path.join(BASE_PATH, "mnb-1.onnx")
with open(onnx_path, "wb") as f:
    f.write(onnx_clf.SerializeToString())

In [39]:
!ls -lah mnb-1.onnx

ls: cannot access 'mnb-1.onnx': No such file or directory


In [40]:
pred_y = clf.predict(val_X)

In [41]:
pred_y[:20]

array(['TGLANG_LANGUAGE_OTHER', 'TGLANG_LANGUAGE_OTHER',
       'TGLANG_LANGUAGE_OTHER', 'TGLANG_LANGUAGE_OTHER',
       'TGLANG_LANGUAGE_OTHER', 'TGLANG_LANGUAGE_GAMS',
       'TGLANG_LANGUAGE_REGEX', 'TGLANG_LANGUAGE_OTHER',
       'TGLANG_LANGUAGE_OTHER', 'TGLANG_LANGUAGE_OTHER',
       'TGLANG_LANGUAGE_OTHER', 'TGLANG_LANGUAGE_OTHER',
       'TGLANG_LANGUAGE_OTHER', 'TGLANG_LANGUAGE_REGEX',
       'TGLANG_LANGUAGE_OTHER', 'TGLANG_LANGUAGE_OTHER',
       'TGLANG_LANGUAGE_OTHER', 'TGLANG_LANGUAGE_OTHER',
       'TGLANG_LANGUAGE_OTHER', 'TGLANG_LANGUAGE_OTHER'], dtype='<U29')

In [42]:
val_y[:20]

980          TGLANG_LANGUAGE_BASIC
1240             TGLANG_LANGUAGE_C
10825        TGLANG_LANGUAGE_OTHER
1281             TGLANG_LANGUAGE_C
10967        TGLANG_LANGUAGE_OTHER
3253          TGLANG_LANGUAGE_GAMS
15944        TGLANG_LANGUAGE_REGEX
9226         TGLANG_LANGUAGE_OTHER
13697        TGLANG_LANGUAGE_REGEX
1862     TGLANG_LANGUAGE_CPLUSPLUS
12214        TGLANG_LANGUAGE_OTHER
12530        TGLANG_LANGUAGE_OTHER
6436           TGLANG_LANGUAGE_SAS
15478        TGLANG_LANGUAGE_REGEX
15610        TGLANG_LANGUAGE_REGEX
14486        TGLANG_LANGUAGE_REGEX
12542        TGLANG_LANGUAGE_OTHER
6367          TGLANG_LANGUAGE_RUST
13667        TGLANG_LANGUAGE_REGEX
11214        TGLANG_LANGUAGE_OTHER
Name: class, dtype: object

In [43]:
!pip3 install onnxruntime

In [44]:
import onnxruntime as rt

In [45]:
sess = rt.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

In [46]:
input_name

'float_input'

In [47]:
label_name

'output_label'

In [48]:
t_val_X = val_X[:1000].toarray().astype('float32')

In [49]:
t_val_X.shape, t_val_X.dtype

((1000, 764896), dtype('float32'))

In [50]:
pred_onx = sess.run([label_name], {input_name: t_val_X})[0]

In [51]:
pred_onx[:20]

array(['TGLANG_LANGUAGE_OTHER', 'TGLANG_LANGUAGE_OTHER',
       'TGLANG_LANGUAGE_OTHER', 'TGLANG_LANGUAGE_OTHER',
       'TGLANG_LANGUAGE_OTHER', 'TGLANG_LANGUAGE_GAMS',
       'TGLANG_LANGUAGE_REGEX', 'TGLANG_LANGUAGE_OTHER',
       'TGLANG_LANGUAGE_OTHER', 'TGLANG_LANGUAGE_OTHER',
       'TGLANG_LANGUAGE_OTHER', 'TGLANG_LANGUAGE_OTHER',
       'TGLANG_LANGUAGE_OTHER', 'TGLANG_LANGUAGE_REGEX',
       'TGLANG_LANGUAGE_OTHER', 'TGLANG_LANGUAGE_OTHER',
       'TGLANG_LANGUAGE_OTHER', 'TGLANG_LANGUAGE_OTHER',
       'TGLANG_LANGUAGE_OTHER', 'TGLANG_LANGUAGE_OTHER'], dtype=object)

In [52]:
equals = pred_onx[:1000] == pred_y[:1000]
sum(equals) == 1000

True

In [53]:
# test_folding = generate_folding(k_folds=K_FOLDS, df=data)

# # k-fold cross validation
# all_scores = []

# for fold_num in tqdm(range(K_FOLDS)):
#     current_test_fold = get_test_fold_for_iteration(test_folding, fold_num + 1)
#     ps = PredefinedSplit(current_test_fold)
#     # breakpoint()
#     # print(pd.Series(current_test_fold).value_counts())
#     scores = cross_val_score(clf, X, y, cv=ps)
#     all_scores.extend(scores)

# print(all_scores)

In [54]:
!pip install scikit-learn onnx onnxruntime skl2onnx

In [60]:
import csv

In [69]:
csv_path = os.path.join(BASE_PATH, "inputs.csv")
with open(csv_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"')
    csvwriter.writerows(t_val_X[:200])

In [70]:
!wc -l {csv_path}

200 /home/sun/Downloads/tg_challenge/inputs.csv


### TODO: per-class evaluation

In [55]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import precision_score, recall_score, f1_score, classification_report


# X = vectorizer.fit_transform(balanced_data['text'])
# y = balanced_data['class']

# train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=42)
# clf = MultinomialNB()
# clf.fit(train_X, train_y)

# pred_y = clf.predict(val_X)

# # precision_per_class = precision_score(val_y, pred_y, average=None)
# # recall_per_class = recall_score(val_y, pred_y, average=None)
# # f1_per_class = f1_score(val_y, pred_y, average=None)

# report = classification_report(val_y, pred_y, output_dict=True)



In [56]:
# report_df = pd.DataFrame(report).T.reset_index().rename(columns={'index': 'class'})
# report_df

# report_df[report_df['class'] == '']
# data['class'].value_counts().sort_values()

In [57]:
# report_df[report_df['class'] == 'TGLANG_LANGUAGE_REGEX'].reset_index()


In [58]:
# for i, row in data.iterrows():
#     if type(row['text']) == list:
#         print(i)

# data.loc[900, 'text']